In [ ]:
%load_ext blackcellmagic

#### Imports and setup

In [ ]:
import deepdish as dd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.formula.api as sm
from statsmodels.stats.multitest import multipletests

In [ ]:
from spacestream.core.constants import SUBJECTS, CORE_ROI_NAMES, ROI_COLORS
from spacestream.core.paths import RESULTS_PATH
from spacestream.utils.general_utils import sem

In [ ]:
seeds = [0,1,2,3,4]
hemis = ["lh","rh"]
model_types = ["MB_RN50_v2", "MB_RN50", "MB_RN18", "TDANN_Supervised", "TDANN_SimCLR"]

#### Load and format data

In [ ]:
# read in data
long = {
    "model_type": [],
    "hemi": [],
    "subject": [],
    "seed": [],
    "ROIS": [],
    "result": [],
}
for hidx, hemi in enumerate(hemis):

    for tidx, mtype in enumerate(model_types):

        if "TDANN" in mtype:
            load_path = (
                RESULTS_PATH
                + "analyses/spatial/TDANNs/optimal_rgb_percentages_top100_"
                + hemi
                + ("_supervised" if "Supervised" in mtype else "")
                + ".hdf"
            )
            sw_key = 5 if "Supervised" in mtype else 2
            data = dd.io.load(load_path)["max_each_stream"]
        else:
            load_path = (
                RESULTS_PATH
                + "analyses/spatial/MBs/"
                + ("RN18" if "18" in mtype else "RN50")
                + "/percent_correct_by_task_"
                + hemi
                + ".hdf"
            )
            data = dd.io.load(load_path)[hemi]

        for sidx, subj in enumerate(SUBJECTS):

            if "TDANN" in mtype:
                for seedx, model_seed in enumerate(seeds):
                    for r in range(len(CORE_ROI_NAMES)):
                        long["model_type"].append(mtype)
                        long["hemi"].append(hemi)
                        long["subject"].append(subj)
                        long["seed"].append(model_seed)
                        long["ROIS"].append(CORE_ROI_NAMES[r])
                        long["result"].append(data[seedx, sidx, sw_key, r] * 100)
            else:
                for r in range(len(CORE_ROI_NAMES)):
                    long["model_type"].append(mtype)
                    long["hemi"].append(hemi)
                    long["subject"].append(subj)
                    long["seed"].append(0)
                    long["ROIS"].append(CORE_ROI_NAMES[r])
                    long["result"].append(data[sidx, r] * 100)

In [ ]:
df = pd.DataFrame(long)
#average across seeds
df_avg = pd.DataFrame(df.groupby(['model_type', 'hemi', 'subject', 'ROIS'])['result'].mean()).reset_index()

In [ ]:
subj2subj = {}
for hidx, hemi in enumerate(hemis):
    load_path = (
        RESULTS_PATH + "analyses/spatial/brains/vox2vox_correspondence_" + hemi + ".hdf"
    )
    subj2subj[hemi] = dd.io.load(load_path)
s2s_hemi_avg = np.mean((subj2subj["lh"], subj2subj["rh"]),axis=0)

In [ ]:
s2s_hemi_avg*100

#### Plotting

In [ ]:
sns.set_theme(style="ticks")
for gidx, group in enumerate(model_types):

    group_df = df_avg[df_avg["model_type"].isin([group])]
    fig, ax = plt.subplots(figsize=(1, 5))
    sns.barplot(
        x="model_type",
        y="result",
        hue="ROIS",
        palette=ROI_COLORS,
        data=group_df,
        ax=ax,
        errorbar="se",
        errwidth=2,
        linewidth=3,
    )
    sns.stripplot(
        x="model_type",
        y="result",
        hue="ROIS",
        palette=ROI_COLORS,
        data=group_df[group_df["hemi"].isin(["lh"])],
        marker="^",
        ax=ax,
        size=5,
        dodge=True,
    )
    sns.stripplot(
        x="model_type",
        y="result",
        hue="ROIS",
        palette=ROI_COLORS,
        data=group_df[group_df["hemi"].isin(["rh"])],
        ax=ax,
        size=5,
        dodge=True,
    )

    for points in ax.collections:

        points.set_edgecolor("white")
        points.set_linewidth(0.75)

    # subj2subj NC bars
    xmin = 0.1
    xmax = 0.28
    for ridx, ROI in enumerate(CORE_ROI_NAMES):
        vals = s2s_hemi_avg[:, ridx] * 100
        y1 = np.mean(vals) - sem(vals)
        y2 = np.mean(vals) + sem(vals)
        ax.axhspan(
            y1,
            y2,
            xmin=xmin + (ridx * 0.26),
            xmax=xmax + (ridx * 0.26),
            alpha=0.9,
            color="lightgray",
        )

    ax.legend([], [], frameon=False)

    ax.set_xlabel("", fontsize=20)
    ax.set_ylabel("Spatial correspondence (%)", fontsize=15)
    ax.tick_params(labelsize=15)
    ax.axhline(33, c="k", linestyle="--")

    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    plt.ylim([0, 80])

    save_path = RESULTS_PATH + "figures/2b_" + group + ".svg"
    fig.savefig(save_path, dpi=100)

#### Statistics

In [ ]:
#set up separate models by ROI
dorsal_df = df_avg[df_avg["ROIS"]=="Dorsal"]
lateral_df = df_avg[df_avg["ROIS"]=="Lateral"]
ventral_df = df_avg[df_avg["ROIS"]=="Ventral"]

In [ ]:
# Dorsal
dorsal_mod = sm.mixedlm('result~model_type', data = dorsal_df, groups=dorsal_df["subject"]).fit()
print(dorsal_mod.summary())

res = pd.concat([dorsal_mod.params,dorsal_mod.pvalues],axis=1)
res.columns=['coefficient','pvalues']
print(res)
res = res[res.index.str.contains('model_type')]
res['corrected_p'] = multipletests(res['pvalues'],method="bonferroni")[1]
print(res)

In [ ]:
# Lateral
lateral_mod = sm.mixedlm('result~model_type', data = lateral_df, groups=lateral_df["subject"]).fit()
print(lateral_mod.summary())

res = pd.concat([lateral_mod.params,lateral_mod.pvalues],axis=1)
res.columns=['coefficient','pvalues']
print(res)

res = res[res.index.str.contains('model_type')]

res['corrected_p'] = multipletests(res['pvalues'],method="bonferroni")[1]
print(res)

In [ ]:
# Ventral
ventral_mod = sm.mixedlm('result~model_type', data = ventral_df, groups=ventral_df["subject"]).fit()
print(ventral_mod.summary())

res = pd.concat([ventral_mod.params,ventral_mod.pvalues],axis=1)
res.columns=['coefficient','pvalues']
print(res)

res = res[res.index.str.contains('model_type')]

res['corrected_p'] = multipletests(res['pvalues'],method="bonferroni")[1]
print(res)